# Extracción de datos desde APIs

## Conceptos básicos de APIs REST y consumo de APIs en Python

### ¿Qué son las API?

- Son un conjuntos de reglas y protocolos que permiten a diferentes aplicaciones comunicarse entre sí. 
- Se utilizan para acceder a datos de servicios web, aplicaciones en la nube, redes sociales, entre otros. 
- Proporcionan una forma estructurada y segura de obtener datos de sistemas externos.

### ![API](../Images/API.png)

### Extracción de datos JSON desde APIs públicas y privadas

### ![API_Spotify](../Images/API_Spotify.png)

# Librerias

In [ ]:
import pandas as pd
import json
import requests
import os
from io import StringIO

# Credenciales

In [ ]:
path = os.getcwd()
path_credentials = os.path.join(path, 'secrets', 'credentials.json')
with open(path_credentials) as file:
    credentials = json.load(file)
    client_id = credentials['client_id']
    client_secret = credentials['client_secret']
    X_api_key = credentials['X_api_key']

# Spotify

In [ ]:
data_spotify = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret
}

token_url = 'https://accounts.spotify.com/api/token'
response = requests.post(token_url, data=data_spotify)

if response.status_code == 200:
    access_token = response.json()['access_token']
else :
    raise Exception(f"Failed to get access token: {response.status_code} {response.text}")

In [ ]:
# id_track = '0bUTHlWbkSQysoM3VsWldT' # Album
id_track = '0dcMqjeDpwqB2xhzMsld0p' # Track
track_url = f'https://api.spotify.com/v1/tracks/{id_track}'
headers = { 'Authorization': f'Bearer {access_token}' }
response = requests.get(track_url, headers=headers)
response.json()

# Mackaroo

Consumo de la API de [mockaroo](https://www.mockaroo.com/), que es una herramienta que permite generar datos de prueba en diferentes formatos, como JSON, CSV, SQL y Excel.

In [ ]:
data_mackaroo = {
    'X-Api-Key': X_api_key,
}
headers = {'X-Api-Key': client_id}

token_url_mackaroo = 'https://my.api.mockaroo.com/mkt_data_cf.json'
response = requests.get(token_url_mackaroo, headers=data_mackaroo)

if response.status_code == 200:
    print(response.json())
else :
    raise Exception(f"Failed to get access token: {response.status_code} {response.text}")

In [ ]:
pd.read_json(StringIO(response.text))

# Ejemplos JSON no estructurados

In [27]:
path_data = os.path.join(os.getcwd(), 'data')

ejemplo1 = os.path.join(path_data, 'ejemplo1.json')
ejemplo2 = os.path.join(path_data, 'ejemplo2.json')
ejemplo3 = os.path.join(path_data, 'ejemplo3.json')

In [11]:
pd.read_json(ejemplo1)

,nombre,productos
0,Tienda de Electrónica,"{'nombre_prod': 'Laptop', 'precio': 800, 'marc..."
1,Tienda de Electrónica,"{'nombre_prod': 'Teléfono', 'precio': 500, 'ma..."


## json_normalize

**¿Qué hace `pd.json_normalize`?**

En esencia, esta función de Pandas toma datos JSON potencialmente complejos y anidados, y los transforma en un DataFrame plano y estructurado, que es mucho más fácil de manipular y analizar.

**Desglose de los parámetros:**

* `data`: Este es el JSON que contiene tus datos.

* `record_path='productos'`: Aquí le estás diciendo a Pandas: "Dentro de mi diccionario `data`, busca una lista llamada 'productos'. Cada elemento dentro de esa lista representa una fila en mi DataFrame final".

* `meta=['nombre']`:  Con esto indicas: "Además de los datos dentro de 'productos', quiero incluir una columna adicional llamada 'nombre' en mi DataFrame. El valor de esta columna será el valor asociado a la clave 'nombre' en el nivel superior de mi diccionario `data`".  En nuestro ejemplo, esto significa que cada fila tendrá el nombre de la tienda ("Tienda de Electrónica") junto con los detalles del producto.


### Ejemplo 1 - JSON con un campo anidado

In [22]:
data = json.load(open(ejemplo1))
pd.json_normalize(
    data 
    , record_path=['productos'] 
    , meta=['nombre']
)

,nombre_prod,precio,marca,nombre
0,Laptop,800,Marca A,Tienda de ElectrÃ³nica
1,TelÃ©fono,500,Marca B,Tienda de ElectrÃ³nica


### Ejemplo 2 - JSON con 2 campos anidados

In [ ]:
data2 = json.load(open(ejemplo2))
pd.json_normalize(
    data2['usuario']
    , record_path=['direcciones']
    , meta=['nombre', 'edad']
)

,calle,ciudad,id,nombre,edad
0,Calle Principal 123,Ciudad A,1,Juan,30
1,Avenida Secundaria 456,Ciudad B,2,Juan,30


In [24]:
pd.json_normalize(
    data2['usuario']
    , record_path=['compras']
    , meta=['nombre', 'edad']
)

,producto,precio,fecha,id,nombre,edad
0,Libro,20,2020-01-01,1,Juan,30
1,PelÃ­cula,15,2020-01-02,2,Juan,30


### Ejemplo 3 - JSON con campos anidados en otros campos anidados

In [35]:
data3 = json.load(open(ejemplo3))
pd.json_normalize(
    data3['escuela']['grados']
    , record_path=['alumnos']
    , meta=['nombre']
    , meta_prefix='grado_'
)

,nombre,edad,grado_nombre
0,Ana,6,Primer Grado
1,Pedro,7,Primer Grado
2,MarÃ­a,7,Segundo Grado
3,Luis,8,Segundo Grado
